# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Import-and-Preprocessing" data-toc-modified-id="Data-Import-and-Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Import and Preprocessing</a></div><div class="lev2 toc-item"><a href="#Distances" data-toc-modified-id="Distances-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Distances</a></div><div class="lev1 toc-item"><a href="#Time-series-kernel" data-toc-modified-id="Time-series-kernel-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Time-series kernel</a></div><div class="lev1 toc-item"><a href="#Spatio-temporal-kernel-with-fixed-variances" data-toc-modified-id="Spatio-temporal-kernel-with-fixed-variances-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Spatio-temporal kernel with fixed variances</a></div><div class="lev1 toc-item"><a href="#Releasing-variances" data-toc-modified-id="Releasing-variances-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Releasing variances</a></div><div class="lev1 toc-item"><a href="#Releasing-all-parameters" data-toc-modified-id="Releasing-all-parameters-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Releasing all parameters</a></div>

Go one step further than in `JuliaGP_spatial3.ipynb` and release the parameters of the time-series kernel.

In [1]:
using TimeSeries
using DataFrames
using GaussianProcesses
using GaussianProcesses: Mean, Kernel, evaluate, metric, IsotropicData, VecF64
using GaussianProcesses: Stationary, KernelData, MatF64
import GaussianProcesses: optimize!, get_optim_target, cov, grad_slice!
import GaussianProcesses: num_params, set_params!, get_params, update_mll!, update_mll_and_dmll!
import GaussianProcesses: get_param_names, cov!, addcov!, multcov!
import Proj4
using Optim
using Distances
;

In [2]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
# plt.rc("figure", figsize=(6,4))
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
;

# Data Import and Preprocessing

In [3]:
include("src/preprocessing.jl")

test_data (generic function with 1 method)

In [4]:
include("src/TempModel.jl")

TempModel

In [5]:
isdList=read_isdList()
isdList[1:5,:]

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,10010,99999,JAN MAYEN(NOR-NAVY),NO,,ENJA,70.933,-8.667,9.0,1931,2015,4.5545e6,6.11344e6
2,10060,99999,EDGEOYA,NO,,,78.25,22.817,14.0,1973,2015,4.04982e6,7.5564e6
3,10070,99999,NY-ALESUND,SV,,,78.917,11.933,7.7,1973,2015,3.8678e6,7.26549e6
4,10080,99999,LONGYEAR,SV,,ENSB,78.246,15.466,26.8,1975,2015,3.99705e6,7.33669e6
5,10090,99999,KARL XII OYA,SV,,,80.65,25.0,5.0,1955,2015,3.69259e6,7.68545e6


In [6]:
isdSubset=isdList[[(usaf in (725450,725460,725480,725485)) for usaf in isdList[:USAF].values],:]
isdSubset

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,725450,14990,THE EASTERN IOWA AIRPORT,US,IA,KCID,41.883,-91.717,264.6,1973,2015,1.64799e6,1.0441e6
2,725460,14933,DES MOINES INTERNATIONAL AIRPORT,US,IA,KDSM,41.534,-93.653,291.7,1973,2015,1.48723e6,1.00379e6
3,725480,94910,WATERLOO MUNICIPAL AIRPORT,US,IA,KALO,42.554,-92.401,264.6,1960,2015,1.59025e6,1.11766e6
4,725485,14940,MASON CITY MUNICIPAL ARPT,US,IA,KMCW,43.154,-93.327,373.4,1973,2015,1.51407e6,1.18374e6


In [7]:
hourly_cat=read_Stations(isdSubset)
hourly_cat[1:5,:]

,year,month,day,hour,min,seconds,temp,ts,station,ts_hours
1,2015,1,1,0,52,0,-7.8,2015-01-01T00:52:00,1,0.866667
2,2015,1,1,1,52,0,-8.3,2015-01-01T01:52:00,1,1.86667
3,2015,1,1,2,52,0,-8.3,2015-01-01T02:52:00,1,2.86667
4,2015,1,1,3,52,0,-9.4,2015-01-01T03:52:00,1,3.86667
5,2015,1,1,4,52,0,-9.4,2015-01-01T04:52:00,1,4.86667


## Distances

To get distances between stations, we can either use a function to compute distances on a sphere, or we can first project the coordinates onto a Euclidean plane, and then compute normal distances. I'll do it both ways to check they're consistent (equal up to a multiplication constant), and then use Euclidean distances for convenience.

In [8]:
# http://www.johndcook.com/blog/python_longitude_latitude/
function distance_on_unit_sphere(lat1, long1, lat2, long2)
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = π/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cosangle = (sin(phi1)*sin(phi2)*cos(theta1 - theta2) +
           cos(phi1)*cos(phi2))
    arc = acos( cosangle )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc
end

distance_on_unit_sphere (generic function with 1 method)

In [9]:
numstations = nrow(isdSubset)
pairwiseSphere = zeros(numstations, numstations)
for i in 1:numstations
    for j in 1:i
        if i==j
            continue
        end
        station1 = isdSubset[i,:]
        station2 = isdSubset[j,:]
        lat1= get(station1[1,:LAT])
        lon1 = get(station1[1,:LON])
        lat2 = get(station2[1,:LAT])
        lon2 = get(station2[1,:LON])
        pairwiseSphere[i,j] = distance_on_unit_sphere(lat1, lon1, lat2, lon2)
        pairwiseSphere[j,i] = pairwiseSphere[i,j]
    end
end
pairwiseSphere

4×4 Array{Float64,2}:
 0.0        0.0259496  0.0146736  0.0303475
 0.0259496  0.0        0.024088   0.0285853
 0.0146736  0.024088   0.0        0.0158124
 0.0303475  0.0285853  0.0158124  0.0      

In [10]:
pairwiseEuclid=pairwise(Euclidean(), Matrix(isdSubset[[:X_PRJ,:Y_PRJ]])')

4×4 Array{Float64,2}:
      0.0        165736.0        93510.4        1.93474e5
 165736.0             0.0            1.53559e5  1.81942e5
  93510.4             1.53559e5      0.0        1.00846e5
      1.93474e5       1.81942e5      1.00846e5  0.0      

Ratio of the two distance matrices: close enough to a constant!

In [11]:
pairwiseEuclid ./ pairwiseSphere

4×4 Array{Float64,2}:
 NaN            6.38684e6    6.37271e6    6.37527e6
   6.38684e6  NaN            6.37493e6    6.36489e6
   6.37271e6    6.37493e6  NaN            6.37765e6
   6.37527e6    6.36489e6    6.37765e6  NaN        

# Time-series kernel

Start with the time-series kernel.

In [12]:
k1 = fix(Periodic(0.0,0.0,log(24.0)), :lp)
k2 = RQIso(0.0,0.0,0.0)
k3 = SEIso(0.0,0.0)
k4 = RQIso(0.0,0.0,0.0)
k5 = RQIso(0.0,0.0,0.0)
k6 = SE(0.0,0.0)
k_time=k1+k2+k3+k4+k5+k6
# hyperparameters fitted in JuliaGP_timeseries_chunks.ipynb
hyp=[-1.46229,-0.0777809,1.03854,1.45757,1.06292,-1.23699,-1.2353,-1.05117,3.10614,1.29327,2.84729,3.67464,0.537794,3.0094,7.70676,-5.30466]
set_params!(k_time, hyp[2:end])

# Spatio-temporal kernel with fixed variances

In [13]:
k_spatial = SEIso(log(2*10^5), log(1.0))
k_means = SEIso(log(10^4), log(10.0))

Type: GaussianProcesses.SEIso, Params: [9.21034,2.30259]


In [14]:
ksp1 = SEIso(log(2*10^5), log(1.0))
ksp2 = SEIso(log(2*10^5), log(1.0))
ksp3 = SEIso(log(2*10^5), log(1.0))
ksp4 = SEIso(log(2*10^5), log(1.0))
ksp5 = SEIso(log(2*10^5), log(1.0))
ksp6 = SEIso(log(2*10^5), log(1.0))
k_spatiotemporal_1 = fix(Masked(k1, [1])) * fix(Masked(ksp1, [2,3]), :lσ) +
                     fix(Masked(k2, [1])) * fix(Masked(ksp2, [2,3]), :lσ) +
                     fix(Masked(k3, [1])) * fix(Masked(ksp3, [2,3]), :lσ) +
                     fix(Masked(k4, [1])) * fix(Masked(ksp4, [2,3]), :lσ) +
                     fix(Masked(k5, [1])) * fix(Masked(ksp5, [2,3]), :lσ) +
                     fix(Masked(k6, [1])) * fix(Masked(ksp6, [2,3]), :lσ) +
                     fix(Masked(k_means, [2,3]))

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.FixedKern, Params: [12.2061]
  Type: GaussianProcesses.FixedKern, Params: Float64[]


In [15]:
# parameters obtained in JuliaGP_spatial3 for the same model
set_params!(k_spatiotemporal_1, [15.4259,9.86868,9.12749,16.4496,15.0163,12.2061])

In [20]:
Optim.Options(x_tol=1e-4)

Optim.Options{Void}(0.0001,1.0e-32,1.0e-8,false,1000,false,false,false,false,1,nothing,NaN)

In [29]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_1
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, -1.0)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    println(reals.dmLL)
    @time opt_out=optimize!(reals, mean=false, show_trace=true, x_tol=1e-5, f_tol=1e-5)
end

[-9192.58,-0.127364,0.339176,3.87699e-19,0.104648,0.258761,1.73226e-6]
Iter     Function value   Gradient norm 
     0     5.131137e+04     9.192581e+03
Base.LinAlg.PosDefException(758)
Base.LinAlg.PosDefException(758)
Base.LinAlg.PosDefException(758)
     1     4.901635e+04     3.545001e+03
     2     4.858904e+04     3.647437e+01
     3     4.858900e+04     3.575821e-01
563.787061 seconds (7.65 G allocations: 233.471 GB, 15.64% gc time)


Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [-1.0,15.4259, ...]
 * Minimizer: [-1.6035385069430337,15.425862451601983, ...]
 * Minimum: 4.858900e+04
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 1.0e-05: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-05: true
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 14
 * Gradient Calls: 8

In [30]:
print(Optim.minimizer(opt_out))

[-1.60354,15.4259,9.86874,9.12749,16.4496,15.0163,12.2061]

In [31]:
print(Optim.minimum(opt_out))

48589.0012141646

In [32]:
print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.kern.σ2)
@printf("l: %5.3f\n", √k1.kern.ℓ2)
@printf("p: %5.0f hours\n", k1.kern.p)
print("> spatial decay:\n")
@printf("l: %5.3f\n", √ksp1.ℓ2)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √ k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f\n", √ksp2.ℓ2)
print("\nk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √k3.σ2)
@printf("l: %5.3f hours\n", √k3.ℓ2)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f\n", √ksp3.ℓ2)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f\n", √ksp4.ℓ2)
print("\nk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k5.σ2)
@printf("l: %5.3f days\n", √k5.ℓ2 / 24)
@printf("α: %5.3f\n",  k5.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f\n", √ksp5.ℓ2)
print("\nk₆ SEIso \n=================\n")
@printf("σ: %5.3f\n", √k6.σ2)
@printf("l: %5.3f days\n", √k6.ℓ2 / 24)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp6.σ2)
@printf("l: %5.3f\n", √ksp6.ℓ2)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(Optim.minimizer(opt_out)[1]))

k₁: Periodic 
σ: 2.825
l: 0.925
p:    24 hours
> spatial decay:
l: 5004571.995

k₂: RQIso 
σ: 2.895
l: 4.296 hours
α: 0.290
> spatial decay:
l: 19317.000

k₃: SEIso 
σ: 0.350
l: 0.291 hours
> spatial decay:
l: 9204.889

k₄: RQIso 
σ: 3.645
l: 0.931 days
α: 17.241
> spatial decay:
l: 13930874.504

k₅: RQIso 
σ: 1.712
l: 1.643 days
α: 20.275
> spatial decay:
l: 3322887.922

k₆ SEIso 
σ: 0.005
l: 92.639 days
> spatial decay:
l: 200005.471

σy: 0.201


# Releasing variances

In [15]:
logNoise = -1.60354
k_spatiotemporal_2 = fix(Masked(k1, [1])) * Masked(ksp1, [2,3]) +
                     fix(Masked(k2, [1])) * Masked(ksp2, [2,3]) +
                     fix(Masked(k3, [1])) * Masked(ksp3, [2,3]) +
                     fix(Masked(k4, [1])) * Masked(ksp4, [2,3]) +
                     fix(Masked(k5, [1])) * Masked(ksp5, [2,3]) +
                     fix(Masked(k6, [1])) * Masked(ksp6, [2,3]) +
                     fix(Masked(k_means, [2,3]))

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [12.2061,0.0]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [12.2061,0.0]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [12.2061,0.0]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [12.2061,0.0]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Params: Float64[]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [12.2061,0.0]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.FixedKern, Param

In [16]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_2
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    @time opt_out=TempModel.optimize_NLopt(reals, mean=false, x_tol=1e-5, f_tol=1e-5)
end

3644.682171 seconds (55.79 G allocations: 1.480 TB, 12.51% gc time)


(47073.84376460839,[-1.65003,13.9813,0.142867,11.5063,-0.081503,9.20607,0.124475,15.2061,0.18125,14.0491,-1.67634,12.2061,-5.11287e-6],:FTOL_REACHED,40)

In [17]:
opt_out

(47073.84376460839,[-1.65003,13.9813,0.142867,11.5063,-0.081503,9.20607,0.124475,15.2061,0.18125,14.0491,-1.67634,12.2061,-5.11287e-6],:FTOL_REACHED,40)

In [19]:
print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.kern.σ2)
@printf("l: %5.3f\n", √k1.kern.ℓ2)
@printf("p: %5.0f hours\n", k1.kern.p)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp1.σ2)
@printf("l: %5.3f\n", √ksp1.ℓ2)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √ k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f\n", √ksp2.ℓ2)
print("\nk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √k3.σ2)
@printf("l: %5.3f hours\n", √k3.ℓ2)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f\n", √ksp3.ℓ2)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f\n", √ksp4.ℓ2)
print("\nk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k5.σ2)
@printf("l: %5.3f days\n", √k5.ℓ2 / 24)
@printf("α: %5.3f\n",  k5.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f\n", √ksp5.ℓ2)
print("\nk₆ SEIso \n=================\n")
@printf("σ: %5.3f\n", √k6.σ2)
@printf("l: %5.3f days\n", √k6.ℓ2 / 24)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp6.σ2)
@printf("l: %5.3f\n", √ksp6.ℓ2)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(opt_out[2][1]))

k₁: Periodic 
σ: 2.825
l: 0.925
p:    24 hours
> spatial decay:
σ: 1.154
l: 1180299.400

k₂: RQIso 
σ: 2.895
l: 4.296 hours
α: 0.290
> spatial decay:
σ: 0.922
l: 99343.470

k₃: SEIso 
σ: 0.350
l: 0.291 hours
> spatial decay:
σ: 1.133
l: 9957.414

k₄: RQIso 
σ: 3.645
l: 0.931 days
α: 17.241
> spatial decay:
σ: 1.199
l: 4017107.385

k₅: RQIso 
σ: 1.712
l: 1.643 days
α: 20.275
> spatial decay:
σ: 0.187
l: 1263149.646

k₆ SEIso 
σ: 0.005
l: 92.639 days
> spatial decay:
σ: 1.000
l: 200000.077

σy: 0.192


We're getting -log-likelihood $48,476$. We've been able to attain $47,082$. Something's not right. [**Update**: That was because we used the ouput of the previous fit as our initial conditions. We once again get to a lower $47,073$. This goes to show how sensitive the optimization is to the initial parameters.]

In [40]:
# I think these are the parameters that previously gave the higher likelihood (47,082)
hyp=[-1.65029,14.2398,0.111707,11.5002,-0.0791469,8.76624,0.126258,14.4041,0.147028,13.0326,-0.635492,12.2061,-8.08864e-7]
set_params!(k_spatiotemporal_2, hyp[2:end])
logNoise=hyp[1]

-1.65029

In [41]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_2
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    println(reals.dmLL)
    @time opt_out=optimize!(reals, mean=false, show_trace=true, x_tol=1e-5, f_tol=1e-5)
end

[-15.7938,-10.1937,25.4768,53.1585,-14.6446,-7.18562e-42,-83.0608,8.93097,5.45921,4.24218,-4.3019,2.16346e-6,-2.87661e-5]
Iter     Function value   Gradient norm 
     0     4.708213e+04     8.306075e+01
Base.LinAlg.PosDefException(338)
     1     4.708200e+04     3.980148e+01
333.347845 seconds (4.92 G allocations: 126.378 GB, 12.79% gc time)


Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [-1.65029,14.2398, ...]
 * Minimizer: [-1.6505292463025218,14.239645584469743, ...]
 * Minimum: 4.708200e+04
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-05: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-05: true
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 6
 * Gradient Calls: 4

In [45]:
print(Optim.minimizer(opt_out))

[-1.65053,14.2396,0.112093,11.501,-0.0793687,8.76624,0.125,14.4042,0.147111,13.0327,-0.635557,12.2061,-8.093e-7]

In [46]:
print(Optim.minimum(opt_out))

47081.9965346642

In [47]:
print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.kern.σ2)
@printf("l: %5.3f\n", √k1.kern.ℓ2)
@printf("p: %5.0f hours\n", k1.kern.p)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp1.σ2)
@printf("l: %5.3f\n", √ksp1.ℓ2)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √ k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f\n", √ksp2.ℓ2)
print("\nk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √k3.σ2)
@printf("l: %5.3f hours\n", √k3.ℓ2)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f\n", √ksp3.ℓ2)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f\n", √ksp4.ℓ2)
print("\nk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k5.σ2)
@printf("l: %5.3f days\n", √k5.ℓ2 / 24)
@printf("α: %5.3f\n",  k5.α)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f\n", √ksp5.ℓ2)
print("\nk₆ SEIso \n=================\n")
@printf("σ: %5.3f\n", √k6.σ2)
@printf("l: %5.3f days\n", √k6.ℓ2 / 24)
print("> spatial decay:\n")
@printf("σ: %5.3f\n", √ksp6.σ2)
@printf("l: %5.3f\n", √ksp6.ℓ2)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(Optim.minimizer(opt_out)[1]))

k₁: Periodic 
σ: 2.825
l: 0.925
p:    24 hours
> spatial decay:
σ: 1.119
l: 1528267.937

k₂: RQIso 
σ: 2.895
l: 4.296 hours
α: 0.290
> spatial decay:
σ: 0.924
l: 98815.055

k₃: SEIso 
σ: 0.350
l: 0.291 hours
> spatial decay:
σ: 1.133
l: 6414.010

k₄: RQIso 
σ: 3.645
l: 0.931 days
α: 17.241
> spatial decay:
σ: 1.158
l: 1801689.308

k₅: RQIso 
σ: 1.712
l: 1.643 days
α: 20.275
> spatial decay:
σ: 0.530
l: 457103.107

k₆ SEIso 
σ: 0.005
l: 92.639 days
> spatial decay:
σ: 1.000
l: 200005.471

σy: 0.192


# Releasing all parameters

In [27]:
logNoise=opt_out[2][1]

-1.6500330136802217

In [30]:
set_params!(k_spatiotemporal_2, 
    [13.9813,0.142867,11.5063,-0.081503,9.20607,0.124475,15.2061,0.18125,14.0491,-1.67634,12.2061,-5.11287e-6])

In [32]:
set_params!(k_time, [-0.0777809,1.03854,1.45757,1.06292,-1.23699,-1.2353,-1.05117,3.10614,1.29327,2.84729,3.67464,0.537794,3.0094,7.70676,-5.30466])

In [33]:
k_spatiotemporal_3 = Masked(k1, [1]) * Masked(ksp1, [2,3]) +
                     Masked(k2, [1]) * Masked(ksp2, [2,3]) +
                     Masked(k3, [1]) * Masked(ksp3, [2,3]) +
                     Masked(k4, [1]) * Masked(ksp4, [2,3]) +
                     Masked(k5, [1]) * Masked(ksp5, [2,3]) +
                     Masked(k6, [1]) * Masked(ksp6, [2,3]) +
                     fix(Masked(k_means, [2,3]))

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.FixedKern}, Params: [-0.0777809,1.03854]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [13.9813,0.142867]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [1.45757,1.06292,-1.23699]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [11.5063,-0.081503]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [-1.2353,-1.05117]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [9.20607,0.124475]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [3.10614,1.29327,2.84729]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [15.2061,0.18125]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQI

In [34]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_3
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    @time opt_out=TempModel.optimize_NLopt(reals, mean=false, x_tol=1e-5, f_tol=1e-5)
end

6168.503799 seconds (59.94 G allocations: 1.590 TB, 9.44% gc time)


(46183.86267863837,[-1.68206,-0.179317,0.945821,13.5116,0.0501475,0.866468,0.758593,-0.984024,11.0867,-0.38583  …  0.129167,3.68457,0.701431,3.00982,14.0459,-1.5127,7.70676,-5.30466,12.2061,-6.18869e-6],:FTOL_REACHED,43)

In [35]:
opt_out

(46183.86267863837,[-1.68206,-0.179317,0.945821,13.5116,0.0501475,0.866468,0.758593,-0.984024,11.0867,-0.38583  …  0.129167,3.68457,0.701431,3.00982,14.0459,-1.5127,7.70676,-5.30466,12.2061,-6.18869e-6],:FTOL_REACHED,43)

In [37]:
print(opt_out[2])

[-1.68206,-0.179317,0.945821,13.5116,0.0501475,0.866468,0.758593,-0.984024,11.0867,-0.38583,-1.44126,-1.13345,9.20607,0.0421904,2.12626,1.24119,-0.15271,15.081,0.129167,3.68457,0.701431,3.00982,14.0459,-1.5127,7.70676,-5.30466,12.2061,-6.18869e-6]